In [4]:
import sqlalchemy
import pandas as pd

In [5]:
from sqlalchemy.engine import URL
connection_url = URL.create(
    "mssql+pyodbc",
    username="rudolfs",
    password="aaa",
    host="localhost",
    port=1433,
    database="balta_bronze",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "TrustServerCertificate": "yes",
    },
)
# engine = sqlalchemy.create_engine('mssql://localhost/balta_bronze?trusted_connection=yes')
connection_url
engine = sqlalchemy.create_engine(connection_url)

In [13]:
# 3.b.1)Python skripts kurš veic unikalitātes identifikāciju polišu piedāvājumu datos (offers). Izveidot šādas dimensijas, pēc rezultāta ieguves izanalizēt vai izveidotais risinājums ir efektīvs:
# UniqeDay – Pēdējais unikālais piedāvājums dienas ietvaros (24h)
# Based on LLM response. Prompt: I have a panadas dataframe with datetime column 'offer_datetime' with multiple records per day. I want to get newest row per day in the dataframe.
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.offers",connection)
    df["offer_datetime"] = pd.to_datetime(df["offer_datetime"])
    newest_per_day = (
        df.loc[
            df.groupby(df_test["offer_datetime"].dt.date)["offer_datetime"]
                   .idxmax()
        ]
        .reset_index(drop=True)
    )
    print("\nNewest Row Per Day")
    print(newest_per_day)


Newest Row Per Day
             offer_id  customer_id product_code product_variant  \
0  O-2024-0007123-D01       100295          PET             Dog   
1      O-2024-0004621       100184          PET             Dog   
2      O-2024-0007571       100678         AUTO           CASCO   
3      O-2024-0006170       100696       TRAVEL           World   
4      O-2024-0000404       100641         AUTO           CASCO   

           offer_datetime  premium_offered sales_source   sum_insured  \
0 2024-01-01 23:27:13.850        85.900002        Phone  24444.630859   
1 2024-01-02 06:42:56.000       166.429993        Agent   6573.450195   
2 2024-01-02 06:40:30.000       231.699997        Phone  47487.179688   
3 2024-01-02 00:27:17.000        71.040001        Phone  31640.000000   
4 2024-01-02 03:27:11.000       101.120003          Web   9678.290039   

                      coverage_hash  
0  74317382b13065bda40e70f4f1e12c8a  
1  dd791fca8a25167ca9d77988c2a91db1  
2  001563e88f774d9b9547e

In [15]:
# 3.b.2)Python skripts kurš veic unikalitātes identifikāciju polišu piedāvājumu datos (offers). Izveidot šādas dimensijas, pēc rezultāta ieguves izanalizēt vai izveidotais risinājums ir efektīvs:
# UniqueWeek - Pēdējais unikālais piedāvājums nedēļas ietvaros (7dienas)
# Based on LLM response. Continued prompt :How would one apply .isocalendar().week to the date before grouping to get the newest record in a week?
import datetime
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.offers",connection)
    df["offer_datetime"] = pd.to_datetime(df["offer_datetime"])
    date_df = df["offer_datetime"].dt.isocalendar()
    df["iso_year"], df["iso_week"] = date_df["year"], date_df["week"]
    
    newest_per_week = (
        df.loc[
            df.groupby(["iso_year", "iso_week"])["offer_datetime"]
                   .idxmax()
        ]
        .reset_index(drop=True)
    )
    print("\nNewest Row Per week")
    print(newest_per_week)


Newest Row Per week
              offer_id  customer_id product_code product_variant  \
0       O-2024-0003525       100339    LIABILITY        Personal   
1       O-2024-0006053       100170         AUTO           CASCO   
2   O-2024-0001151-D01       100879         AUTO            MTPL   
3   O-2024-0004922-D01       100855    LIABILITY        Personal   
4       O-2024-0000498       100869         AUTO            MTPL   
..                 ...          ...          ...             ...   
77  O-2025-0001620-D01       100739         AUTO            MTPL   
78  O-2025-0006917-D01       100608         HOME        HomePlus   
79  O-2025-0007445-D01       100232       TRAVEL          Europe   
80  O-2025-0000249-D01       100585         AUTO            MTPL   
81  O-2025-0004561-D01       100750         AUTO            MTPL   

            offer_datetime  premium_offered sales_source   sum_insured  \
0  2024-01-07 23:25:42.000       119.010002        Agent   8627.280273   
1  2024-01-14 

In [18]:
# UniqueMonth- Pēdējais unikālais piedāvājums mēneša ietvaros(30dienas);
# Based on LLM response. Continued prompt 


import datetime
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT *  FROM balta_bronze.dbo.offers",connection)
    df["offer_datetime"] = pd.to_datetime(df["offer_datetime"])
    date_df = df["offer_datetime"].dt.isocalendar()
    df["ym"] = df["offer_datetime"].dt.to_period("M")    
    print(df.head())
    newest_per_month = (
        df.loc[
            df.groupby(["ym"])["offer_datetime"]
                   .idxmax()
        ]
        .reset_index(drop=True)
    )
    print("\nNewest Row Per month")
    print(newest_per_month)

         offer_id  customer_id product_code product_variant  \
0  O-2024-0006081       100071       TRAVEL          Europe   
1  O-2024-0007701       100870       TRAVEL           World   
2  O-2024-0002102       100159       TRAVEL          Europe   
3  O-2024-0003238       100463       TRAVEL           World   
4  O-2024-0006953       100591          PET             Cat   

       offer_datetime  premium_offered sales_source   sum_insured  \
0 2024-01-01 01:36:31        39.790001        Agent  34246.839844   
1 2024-01-01 02:45:03        53.150002          Web  40268.031250   
2 2024-01-01 06:00:28        27.180000        Agent  15595.240234   
3 2024-01-01 06:02:03        57.240002          Web  36560.820312   
4 2024-01-01 07:54:43        70.000000        Phone  21046.679688   

                      coverage_hash       ym  
0  8929389deeeb0611096b6e99cc59462e  2024-01  
1  29228477efa77b80adf72d52898ed504  2024-01  
2  585fbb6778feb0b34d9d9968379153d5  2024-01  
3  1b416383d03705a